# Sistema de ecuaciones diferenciales parcials de adveccion-difusion-reaccion

El problema es el siguiente sistema de EDPs:
$$\begin{cases} \dfrac{\partial u_1}{\partial t}+w\cdot\nabla u_1-\nabla\cdot(\epsilon\nabla u_1)=f_1-Ku_1u_2, \\ \dfrac{\partial u_2}{\partial t}+w\cdot\nabla u_2-\nabla\cdot(\epsilon\nabla u_2)=f_2-Ku_1u_2  \\ \dfrac{\partial u_3}{\partial t}+w\cdot\nabla u_3-\nabla\cdot(\epsilon\nabla u_3)=f_2+Ku_1u_2-Ku_3\end{cases}.$$

\Este sistema modela la reaccion quimica entre dos sustancias $A$ y $B$ en algun dominio $\Omega$:$$A+B\rightarrow C.$$

Asumimos que la reaccion es de primer orden (i.e., las concentraciones de $[A]$ y $[B]$ son proporcionales de las dos sustancias $A$ y $B$):$$\dfrac{d}{dt}[C]=K[A][B].$$

Tambien asumimos que la sustancia formada $C$  espontaneamente decae   con una tasa proporcional debido a la concentracion de $C$. En la EDP anterior denotamos las concentraciones $u_1=[A]$, $u_2=[B]$ y $u_3=[C]$.

La reaccion quimica se produce en cada punto del dominio $\Omega$. Ademas, asumimos que las sustancias $A$, $B$ y $C$ se difunden enn el dominio $\Omega$ con constante de fidusion $\epsilon$ (los terminos $-\nabla(\epsilon\nabla u_i)$) y son transportados con velocidad $w$ (los terminos $w\cdot\nabla u_i$).

Para hacer las cosas mas interesantes y ponernos en un problema "real", dejaremos que la reacción química tenga lugar en el campo de velocidad calculado a partir de la solución de las ecuaciones incompresibles de Navier-Stokes alrededor de un cilindro. En resumen, estaremos resolviendo así el siguiente sistema acoplado de EPDs no lineales:
$$\begin{cases} \varsigma\left(\dfrac{\partial}{\partial t}+w\cdot\nabla w\right)=\nabla\cdot\varsigma(w,p)+f \\ \nabla\cdot w=0 \\ \dfrac{\partial u_1}{\partial t}+w\cdot\nabla u_1-\nabla\cdot(\epsilon\nabla u_1)=f_1-Ku_1u_2, \\ \dfrac{\partial u_2}{\partial t}+w\cdot\nabla u_2-\nabla\cdot(\epsilon\nabla u_2)=f_2-Ku_1u_2  \\ \dfrac{\partial u_3}{\partial t}+w\cdot\nabla u_3-\nabla\cdot(\epsilon\nabla u_3)=f_2+Ku_1u_2-Ku_3.\end{cases}$$

Suponemos que Las cocnentraciones en $t=0$ son cero e introducimos las sustancias $A$ y $B$ en el sistema mediantes las fuentes no nulas $f_1$ y $f_2$ cerca de las esquinas en la entrada, y tomamos $f-3=0$. El resultado será que $A$ y $B$ son convectivos por advección y difusión a lo largo del canal, y cuando se mezclen se formarán la sustancia $C$.

## Formulacion Variacional



## Implementacion

In [30]:
from dolfin import *

import matplotlib.pyplot as plt

In [31]:
T = 5.0 # tiempo final
num_steps = 500 # numero de pasos
dt = T / num_steps # tamano del paso temporal
eps = 0.01 # coeficiente de difusion
K = 10.0 # razon de reaccion

In [32]:
# Cargamos la malla
mesh = Mesh('navier_stokes_cylinder/cylinder.xml.gz')

In [33]:
# Definimos el espacio funcional de velocidad
W = VectorFunctionSpace(mesh, 'P', 2)

In [34]:
# Definimos el espacio funcional  para el sistema de concetraciones
P1 = FiniteElement('P', triangle, 1)
element = MixedElement([P1, P1, P1])
V = FunctionSpace(mesh, element)

In [35]:
# Definimos las funciones test
v_1, v_2, v_3 = TestFunction(V)

In [36]:
# Definimos las funciones para velocidad y las concentraciones
w = Function(W)
u = Function(V)
u_n = Function(V)

In [37]:
# Dividimos el sistema de funciones para acceder a sus componentes
u_1, u_2, u_3 = split(u)
u_n1, u_n2, u_n3 = split(u_n)

In [38]:
# Definimos las fuentes
f_1 = Expression('pow(x[0] - 0.1, 2) + pow(x[1] - 0.1, 2) < 0.05 * 0.05 ? 0.1 : 0', degree = 1)
f_2 = Expression('pow(x[0] - 0.1, 2) + pow(x[1] - 0.3, 2) < 0.05 * 0.05 ? 0.1 : 0', degree = 1)
f_3 = Constant(0)

In [39]:
# Definimos expresiones usadas en la formulacion Variacional
k = Constant(dt)
K = Constant(K)
eps = Constant(eps)

In [40]:
# Definimos el problema variacional
F = ((u_1 - u_n1) / k)*v_1*dx + dot(w, grad(u_1))*v_1*dx \
    + eps*dot(grad(u_1), grad(v_1))*dx + K*u_1*u_2*v_1*dx \
    + ((u_2 - u_n2) / k)*v_2*dx + dot(w, grad(u_2))*v_2*dx \
    + eps*dot(grad(u_2), grad(v_2))*dx + K*u_1*u_2*v_2*dx \
    + ((u_3 - u_n3) / k)*v_3*dx + dot(w, grad(u_3))*v_3*dx \
    + eps*dot(grad(u_3), grad(v_3))*dx - K*u_1*u_2*v_3*dx + K*u_3*v_3*dx \
    - f_1*v_1*dx - f_2*v_2*dx - f_3*v_3*dx

In [41]:
# Creamos una serie de tiempo para leer la data de velocidad
timeseries_w = TimeSeries('navier_stokes_cylinder/velocity_series.h5')

In [42]:
# Creamos archivos VTK para la visualizacion
vtkfile_u_1 = File('reaction_system/u_1.pvd')
vtkfile_u_2 = File('reaction_system/u_2.pvd')
vtkfile_u_3 = File('reaction_system/u_3.pvd')

In [43]:
# Creamos una barra de progreso
progress = Progress('Time-stepping')
#set_log_level(progress)

In [45]:
# Calculo de la solucion numerica
t = 0
for n in range(num_steps):
    # Actualziacion del tiempo
    t += dt

    # Leemos la velocidad
    #timeseries_w.retrieve(w.vector(), t)

    # Resolvemos el problema variacional para cada paso
    solve(F == 0, u)

    # Guardamos la solucion
    _u_1, _u_2, _u_3 = u.split()

    vtkfile_u_1 << (_u_1, t)
    vtkfile_u_2 << (_u_2, t)
    vtkfile_u_3 << (_u_3, t)

    # Actualizamos la solucion previa
    u_n.assign(u)

    # Actualizamos la barra de progreso 
    #progress.update(t / T)
    #progress += t/T
# Hold plot
#plt.show()


No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 9.837e-19 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton solver finished in 0 iterations and 0 linear solver iterations.
No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 9.895e-05 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 4.797e-10 (tol = 1.000e-10) r (rel) = 4.848e-06 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 1.044e-18 (tol = 1.000e-10) r (rel) = 1.056e-14 (tol = 1.000e-09)
  Newton solver finished in 2 iterations and 2 linear solver iterations.
No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear var